In [ ]:
import requests
import pathlib
import pandas as pd
import time

# A function to fetch weather data for a given city
def fetch_weather_data(city):
    url = "https://archive-api.open-meteo.com/v1/archive"
    # The parameters for the API request
    params = {
        "latitude": city["latitude"],
        "longitude": city["longitude"],
        "start_date": "2010-01-01",
        "end_date": "2024-07-06",
        "daily": "weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration",
        "timezone": "GMT",
        "min": "2010-01-01",
        "max": "2024-07-06",
    }
# The number of retries and the backoff factor for rate limiting
    retries = 5
    backoff_factor = 1

# Attempt to fetch data from the API
    for attempt in range(retries):
        try:
            print(f"Querying weather API for {city['name']} - {city['country']} (Attempt {attempt + 1})...")
            res = requests.get(url, params=params)
            res.raise_for_status()
            return res.json()
        except requests.exceptions.HTTPError as http_err:
            if res.status_code == 429: # Check if rate limit exceeded
                wait_time = backoff_factor * (2 ** attempt)
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"HTTP error occurred: {http_err}")
                break
        except Exception as err:
            print(f"Other error occurred: {err}")
            break

    return None

# The main function to orchestrate the data fetching and saving process

def main():
    cities = [
        { "name": "Dar es Salaam", "country": "Tanzania", "latitude": -6.8235, "longitude": 39.2695},
    ]

    cities_dfs = [] # List to store dataframes for each city
    # Iterate over each city to fetch and process the weather data
    for city in cities:
        data = fetch_weather_data(city)
        if data: # Check if data was successfully retrieved
            print("Preprocessing...")
            df = pd.DataFrame(data["daily"])
            df["latitude"] = data["latitude"]
            df["longitude"] = data["longitude"]
            df["elevation"] = data["elevation"]
            df["country"] = city["country"]
            df["city"] = city["name"]

            cities_dfs.append(df)
        else:
            print(f"Failed to retrieve data for {city['name']} - {city['country']}")

    if cities_dfs:
        concat_df = pd.concat(cities_dfs, ignore_index=True)
        file_name = "open_meteo.csv" # the output file name
        concat_df.to_csv(file_name, index=False)
        print(f"Data saved to {file_name}")
    else:
        print("No data to save.")

if __name__ == "__main__":
    main()


Querying weather API for Dar es Salaam - Tanzania (Attempt 1)...
Preprocessing...
Data saved to open_meteo.csv
